Gold nanoparticle rotation
---

## Libraries

In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R

## Canonical rotation matrix <a name="S1"></a> 

$
\mathbf{R}_{xyz} = \mathbf{R}_{x}\mathbf{R}_{y}\mathbf{R}_{z} = \left[R_{ij}\right] =
$ 

\begin{bmatrix}
c_y c_z & -c_y s_z & s_y \\
s_x s_y c_z + c_x s_z & -s_x s_y c_z + c_x c_z & -s_x c_y \\
-c_x s_y c_z + s_x s_z & c_x s_y s_z + s_x c_z & c_x c_y 
\end{bmatrix}

$
\text{where } c_i = \cos\theta_i,\; s_i = \sin\theta_i,\;\text{e } i=x,y,z.
$

so if I know the matrix $\mathbf{R}_{xyz}$ I can find the angles of rotation:

$ \theta_y = \sin^{-1}\left( R_{1,3} \right)$

$ \theta_z = \cos^{-1}\left(  \dfrac{R_{1,1}}{\sqrt{1-R_{1,3}^2}} \right)$

$ \theta_x = \sin^{-1}\left(  -\dfrac{R_{2,3}}{\sqrt{1-R_{1,3}^2}}  \right)$

In [2]:
def R2theta(R):
    theta_y = np.arcsin(R[0, 2])
    theta_z = np.arccos(R[0, 0] * (1 - R[0, 2] ** 2) ** (-1 / 2))
    theta_x = np.arcsin(-R[1, 2] * (1 - R[0, 2] ** 2) ** (-1 / 2))
    return theta_x, theta_y, theta_z

`scipy` has the function to calculate the rotational matrix, it wants as input the corresponding ([quaternion](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation)).

If you know the axis $\mathbf{r}$ and the rotation angle $\theta$ the quaternion $\mathbf{q}$ is:

$
\mathbf{q} = \left\{ \cos\theta, r_x\cdot\sin\theta,  r_y\cdot\sin\theta, r_z\cdot\sin\theta \right\}^T
$

In [6]:
r = np.r_[0, -(2 ** (-0.5)), 2 ** (-0.5)]  # axis of rotation
theta = np.arccos(3 ** (-0.5))  # rotation angle
q = np.r_[np.cos(theta / 2), r * np.sin(theta / 2)]  # corresponding quaternion

R_u = R.from_quat(q).as_matrix()

In [7]:
theta_x, theta_y, theta_z = R2theta(R_u)

print(
    "θx = {:1.16f} rad,\nθy = {:1.16f} rad,\nθz = {:1.16f} rad.".format(
        theta_x, theta_y, theta_z
    )
)

θx = 0.7853981633974482 rad,
θy = -0.6154797086703873 rad,
θz = 0.2617993877991497 rad.


We calculate the configurations

In [11]:
N = 4
theta = np.linspace(0, np.arccos(3 ** (-0.5)), N)

for i, theta_i in enumerate(theta):
    q = np.r_[np.cos(theta_i / 2), r * np.sin(theta_i / 2)]
    R_u = R.from_quat(q).as_matrix()
    theta_x, theta_y, theta_z = R2theta(R_u)
    print("-- Case %d " % (i + 1) + "-" * 70)
    print(
        "   ┌ %+1.3f ┐                   ┌ %+1.3f %+1.3f %+1.3f ┐"
        % (r[0], R_u[0, 0], R_u[0, 1], R_u[0, 2])
    )
    print(
        "r =│ %+1.3f │, θ = %1.3f --> R =│ %+1.3f %1.3f %+1.3f │ "
        % (r[1], theta_i, R_u[1, 0], R_u[1, 1], R_u[1, 2])
    )
    print(
        "   └ %+1.3f ┘                   └ %+1.3f %+1.3f %+1.3f ┘"
        % (r[2], R_u[2, 0], R_u[2, 1], R_u[2, 2])
    )
    print("\nANGOLI x, y, z:")
    print(
        "θx = %1.16f rad \tθx = %1.16f deg,\nθy = %1.16f rad \tθy = %1.16f deg,\nθz = %1.16f rad \tθz = %1.16f deg.\n"
        % (
            theta_x,
            np.rad2deg(theta_x),
            theta_y,
            np.rad2deg(theta_y),
            theta_z,
            np.rad2deg(theta_z),
        )
    )

-- Case 1 ----------------------------------------------------------------------
   ┌ +0.000 ┐                   ┌ +1.000 +0.000 +0.000 ┐
r =│ -0.707 │, θ = 0.000 --> R =│ +0.000 -1.000 -0.000 │ 
   └ +0.707 ┘                   └ -0.000 +0.000 -1.000 ┘

ANGOLI x, y, z:
θx = 0.0000000000000000 rad 	θx = 0.0000000000000000 deg,
θy = 0.0000000000000000 rad 	θy = 0.0000000000000000 deg,
θz = 0.0000000000000000 rad 	θz = 0.0000000000000000 deg.

-- Case 2 ----------------------------------------------------------------------
   ┌ +0.000 ┐                   ┌ +0.975 +0.025 -0.221 ┐
r =│ -0.707 │, θ = 0.318 --> R =│ -0.025 -0.975 -0.221 │ 
   └ +0.707 ┘                   └ -0.221 +0.221 -0.950 ┘

ANGOLI x, y, z:
θx = 0.2290136875905215 rad 	θx = 13.1215177496644380 deg,
θy = -0.2232334906601559 rad 	θy = -12.7903368608000143 deg,
θz = 0.0257797964566727 rad 	θz = 1.4770735336736598 deg.

-- Case 3 ----------------------------------------------------------------------
   ┌ +0.000 ┐            